##### Copyright 2019 The TensorFlow Authors.

In [8]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [9]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [10]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [12]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.24573427, -0.2827672 , -0.00430182, -0.12439328,  0.07742131,
        -0.6023559 , -0.11142676, -0.5992842 ,  0.02324222, -0.50861824]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [13]:
tf.nn.softmax(predictions).numpy()

array([[0.09642611, 0.09292049, 0.12275734, 0.10886604, 0.13321078,
        0.06750189, 0.11028684, 0.06770955, 0.12618557, 0.0741354 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [15]:
loss_fn(y_train[:1], predictions).numpy()

2.6955998

In [16]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [17]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 836us/step - loss: 0.2928 - accuracy: 0.9151
Epoch 2/5
1875/1875 [==============================] - 2s 850us/step - loss: 0.1395 - accuracy: 0.9585
Epoch 3/5
1875/1875 [==============================] - 2s 845us/step - loss: 0.1041 - accuracy: 0.9684
Epoch 4/5
1875/1875 [==============================] - 2s 846us/step - loss: 0.0863 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 2s 842us/step - loss: 0.0734 - accuracy: 0.9769


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [18]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0714 - accuracy: 0.9779


[0.07142401486635208, 0.9779000282287598]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [19]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [20]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.39350760e-07, 5.61585409e-08, 8.06640401e-06, 1.95387824e-04,
        2.17979038e-11, 8.63096318e-07, 1.49840871e-12, 9.99788702e-01,
        1.55338819e-06, 5.20440199e-06],
       [4.37444192e-10, 2.81791697e-04, 9.99690771e-01, 2.46028249e-05,
        4.23417235e-14, 2.33224484e-09, 2.11959561e-09, 3.14547659e-11,
        2.87286230e-06, 9.15676565e-14],
       [3.70627987e-07, 9.99834418e-01, 2.32873717e-05, 8.39260053e-07,
        3.12203224e-06, 1.28250565e-06, 2.74235185e-06, 1.13488772e-04,
        2.03278851e-05, 2.68692570e-08],
       [9.99963522e-01, 1.05154045e-08, 9.06049536e-06, 1.22822300e-07,
        1.42912768e-07, 1.13717348e-07, 1.26525238e-05, 1.37785655e-05,
        1.34224294e-07, 5.17354692e-07],
       [6.02121472e-05, 1.79602271e-08, 1.88446356e-05, 1.30588873e-07,
        9.89090502e-01, 7.23571134e-07, 4.79346500e-06, 3.56597244e-04,
        8.12782582e-06, 1.04600759e-02]], dtype=float32)>